<a href="https://colab.research.google.com/github/valentina-992/MLlib/blob/main/M9S5_AP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ACTIVIDAD SESIÓN INTRODUCIÓN A MACHINE LEARNING ESCALABLE
Una tienda de cosmética quiere desarrollar un sistema inteligente que clasifique productos de **skin care** en diferentes categorías según sus características. La clasificación ayudará a recomendar
productos adecuados a los clientes según su tipo de piel.

La tienda proporciona un dataset con información de productos, incluyendo:
- **Ingredientes clave** (como ácido hialurónico, retinol, vitamina C, etc.)
- **Nivel de hidratación**
- **Nivel de absorción**
- **Factor de protección solar (SPF)**
- **Tipo de piel recomendado** (seco, graso, mixto o sensible)

El objetivo es entrenar un modelo de clasificación con MLlib que prediga el tipo de piel recomendado para cada producto.

**Importante:**
Se debe convertir **Tipo de Piel** en valores numéricos:
- Seco → 0
- Graso → 1
- Mixto → 2
- Sensible → 3

**INSTRUCCIONES**
## 1. Carga y exploración de datos (2 puntos)
- Cargar los datos desde **skincare_products.csv** en un DataFrame de PySpark.
- Mostrar las primeras filas del dataset.
- Realizar un resumen estadístico de las variables numéricas.

In [ ]:
# Librerías
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from google.colab import files

In [ ]:
# Carga archivo
files.upload()

Saving skincare_products.csv to skincare_products.csv


{'skincare_products.csv': b'Ingredientes,Hidrataci\xc3\xb3n,Absorci\xc3\xb3n,SPF,Tipo de Piel\r\n\xc3\x81cido Hialur\xc3\xb3nico,Alto,Medio,0,Seco\r\nRetinol,Bajo,Alto,0,Graso\r\nVitamina C,Medio,Medio,30,Mixto\r\nAloe Vera,Alto,Bajo,15,Sensible\r\nNiacinamida,Medio,Medio,0,Mixto\r\nCeramidas,Alto,Bajo,0,Seco\r\n\xc3\x81cido Salic\xc3\xadlico,Bajo,Alto,0,Graso\r\nCentella Asi\xc3\xa1tica,Medio,Medio,20,Sensible\r\nExtracto de T\xc3\xa9 Verde,Medio,Alto,0,Mixto\r\nManteca de Karit\xc3\xa9,Alto,Bajo,0,Seco\r\nExtracto de Regaliz,Medio,Medio,15,Sensible\r\nVitamina E,Alto,Medio,25,Seco\r\nBakuchiol,Bajo,Medio,0,Mixto\r\n\xc3\x81cido K\xc3\xb3jico,Medio,Alto,0,Graso\r\nBeta-Glucanos,Alto,Bajo,10,Sensible\r\nResveratrol,Medio,Medio,20,Mixto\r\nP\xc3\xa9ptidos,Bajo,Alto,0,Graso\r\nEscualano,Alto,Medio,0,Seco\r\nArbutina,Medio,Alto,0,Mixto\r\nUrea,Alto,Bajo,15,Sensible\r\n'}

In [ ]:
# Sesión spark
spark = SparkSession.builder.appName("SkincareProducts").getOrCreate()
sc = spark.sparkContext

In [ ]:
# Creación de DataFrame con spark
df = spark.read.csv("skincare_products.csv", header=True, inferSchema=True, encoding='utf-8')

# Mostrar primeros registros
df.show()

+--------------------+-----------+---------+---+------------+
|        Ingredientes|Hidratación|Absorción|SPF|Tipo de Piel|
+--------------------+-----------+---------+---+------------+
|   Ácido Hialurónico|       Alto|    Medio|  0|        Seco|
|             Retinol|       Bajo|     Alto|  0|       Graso|
|          Vitamina C|      Medio|    Medio| 30|       Mixto|
|           Aloe Vera|       Alto|     Bajo| 15|    Sensible|
|         Niacinamida|      Medio|    Medio|  0|       Mixto|
|           Ceramidas|       Alto|     Bajo|  0|        Seco|
|    Ácido Salicílico|       Bajo|     Alto|  0|       Graso|
|   Centella Asiática|      Medio|    Medio| 20|    Sensible|
|Extracto de Té Verde|      Medio|     Alto|  0|       Mixto|
|   Manteca de Karité|       Alto|     Bajo|  0|        Seco|
| Extracto de Regaliz|      Medio|    Medio| 15|    Sensible|
|          Vitamina E|       Alto|    Medio| 25|        Seco|
|           Bakuchiol|       Bajo|    Medio|  0|       Mixto|
|       

In [ ]:
# Resumen estadístico de variables numéricas
df.select("Hidratación", "Absorción", "SPF").describe().show()

+-------+-----------+---------+-----------------+
|summary|Hidratación|Absorción|              SPF|
+-------+-----------+---------+-----------------+
|  count|         20|       20|               20|
|   mean|       NULL|     NULL|              7.5|
| stddev|       NULL|     NULL|10.19545822516343|
|    min|       Alto|     Alto|                0|
|    max|      Medio|    Medio|               30|
+-------+-----------+---------+-----------------+



## 2. Preprocesamiento de datos (2 puntos)
- Convertir la columna "Tipo de Piel" en valores numéricos (0, 1, 2, 3).
- Transformar las variables categóricas (Hidratación y Absorción) en valores numéricos.
  - Hidratación: Bajo (0), Medio (1), Alto (2)
  - Absorción: Bajo (0), Medio (1), Alto (2)
- Unir todas las características en un vector usando **VectorAssembler**.

In [ ]:
# Transformación Tipo de Piel
# Seco = 0, Graso = 1, Mixto = 2, Sensible = 3
df = df.withColumn("Tipo de Piel Index",
                   (when(col("Tipo de Piel") == "Seco", 0)
                    .when(col("Tipo de Piel") == "Graso", 1)
                    .when(col("Tipo de Piel") == "Mixto", 2)
                    .when(col("Tipo de Piel") == "Sensible", 3)
                    .otherwise(None)))

# Eliminar columna tipo de piel
df = df.drop("Tipo de Piel")
df.show()


+--------------------+-----------+---------+---+------------------+
|        Ingredientes|Hidratación|Absorción|SPF|Tipo de Piel Index|
+--------------------+-----------+---------+---+------------------+
|   Ácido Hialurónico|       Alto|    Medio|  0|                 0|
|             Retinol|       Bajo|     Alto|  0|                 1|
|          Vitamina C|      Medio|    Medio| 30|                 2|
|           Aloe Vera|       Alto|     Bajo| 15|                 3|
|         Niacinamida|      Medio|    Medio|  0|                 2|
|           Ceramidas|       Alto|     Bajo|  0|                 0|
|    Ácido Salicílico|       Bajo|     Alto|  0|                 1|
|   Centella Asiática|      Medio|    Medio| 20|                 3|
|Extracto de Té Verde|      Medio|     Alto|  0|                 2|
|   Manteca de Karité|       Alto|     Bajo|  0|                 0|
| Extracto de Regaliz|      Medio|    Medio| 15|                 3|
|          Vitamina E|       Alto|    Medio| 25|

In [ ]:
# Transformación Hidratación y Absorción
df = df.withColumn("Hidratación Index",
                   (when(col("Hidratación") == "Bajo", 0)
                    .when(col("Hidratación") == "Medio", 1)
                    .when(col("Hidratación") == "Alto", 2)
                    .otherwise(None)))

df = df.withColumn("Absorción Index",
                   (when(col("Absorción") == "Bajo", 0)
                    .when(col("Absorción") == "Medio", 1)
                    .when(col("Absorción") == "Alto", 2)
                    .otherwise(None)))

# Drop the original columns and update the DataFrame
df = df.drop("Hidratación", "Absorción")

df.show()

+--------------------+---+------------------+-----------------+---------------+
|        Ingredientes|SPF|Tipo de Piel Index|Hidratación Index|Absorción Index|
+--------------------+---+------------------+-----------------+---------------+
|   Ácido Hialurónico|  0|                 0|                2|              1|
|             Retinol|  0|                 1|                0|              2|
|          Vitamina C| 30|                 2|                1|              1|
|           Aloe Vera| 15|                 3|                2|              0|
|         Niacinamida|  0|                 2|                1|              1|
|           Ceramidas|  0|                 0|                2|              0|
|    Ácido Salicílico|  0|                 1|                0|              2|
|   Centella Asiática| 20|                 3|                1|              1|
|Extracto de Té Verde|  0|                 2|                1|              2|
|   Manteca de Karité|  0|              

In [ ]:
# Esquema
df.printSchema()

root
 |-- Ingredientes: string (nullable = true)
 |-- SPF: integer (nullable = true)
 |-- Tipo de Piel Index: integer (nullable = true)
 |-- Hidratación Index: integer (nullable = true)
 |-- Absorción Index: integer (nullable = true)



In [ ]:
# Unión de características con VectorAssembler
assembler = VectorAssembler(inputCols=["Hidratación Index", "Absorción Index", "SPF"], outputCol="features")
df = assembler.transform(df)
df.show()

+--------------------+---+------------------+-----------------+---------------+--------------+
|        Ingredientes|SPF|Tipo de Piel Index|Hidratación Index|Absorción Index|      features|
+--------------------+---+------------------+-----------------+---------------+--------------+
|   Ácido Hialurónico|  0|                 0|                2|              1| [2.0,1.0,0.0]|
|             Retinol|  0|                 1|                0|              2| [0.0,2.0,0.0]|
|          Vitamina C| 30|                 2|                1|              1|[1.0,1.0,30.0]|
|           Aloe Vera| 15|                 3|                2|              0|[2.0,0.0,15.0]|
|         Niacinamida|  0|                 2|                1|              1| [1.0,1.0,0.0]|
|           Ceramidas|  0|                 0|                2|              0| [2.0,0.0,0.0]|
|    Ácido Salicílico|  0|                 1|                0|              2| [0.0,2.0,0.0]|
|   Centella Asiática| 20|                 3|     

## 3. División de datos y entrenamiento del modelo (3 puntos)
- Dividir los datos en 80% entrenamiento y 20% prueba.
- Entrenar un modelo de Árboles de Decisión con MLlib usando las características del **dataset**.

In [ ]:
# División de datos
(train_data, test_data) = df.randomSplit([0.8, 0.2], seed=42)

In [ ]:
# Creación de modelo Árbol de Decisión
dt = DecisionTreeClassifier(labelCol="Tipo de Piel Index", featuresCol="features")

# Entrenamiento de modelo
model = dt.fit(train_data)

## 4. Predicción y evaluación (2 puntos)
- Aplicar el modelo al conjunto de prueba y mostrar las predicciones.
- Calcular la precisión del modelo usando **MulticlassClassificationEvaluator**.

In [ ]:
# Aplicar el modelo a conjunto de prueba
predictions = model.transform(test_data)
predictions.select("features", "Tipo de Piel Index", "prediction").show()

+-------------+------------------+----------+
|     features|Tipo de Piel Index|prediction|
+-------------+------------------+----------+
|[0.0,1.0,0.0]|                 2|       2.0|
|[2.0,1.0,0.0]|                 0|       0.0|
|[1.0,2.0,0.0]|                 2|       1.0|
|[0.0,2.0,0.0]|                 1|       1.0|
|[0.0,2.0,0.0]|                 1|       1.0|
+-------------+------------------+----------+



In [ ]:
# Precisión del modelo
evaluator = MulticlassClassificationEvaluator(labelCol="Tipo de Piel Index", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Precisión del modelo: {:.2f}%".format(accuracy * 100))

Precisión del modelo: 80.00%


## 5. Análisis de resultados y mejoras (1 punto)
- Explicar en breve (3-5 líneas) qué tan preciso fue el modelo y cómo se podría mejorar (por
ejemplo, usando otro algoritmo o ajustando parámetros).

El modelo alcanzó una precisión de un 80%, es decir, logra clasificar correctamente la mayoría de los productos de acuerdo al tipo de piel. Podríamos mejorar esta métrica probando otros métodos más robustos como RandomForestClassifier, además de utilizar validación cruzada o grid search.

# EXTRAS

In [ ]:
# Prueba con random forest
rf = RandomForestClassifier(labelCol="Tipo de Piel Index", featuresCol="features")

paramGrid = (ParamGridBuilder()
             .addGrid(rf.numTrees, [10, 20, 50])         # Número de árboles
             .addGrid(rf.maxDepth, [5, 10])              # Profundidad máxima
             .addGrid(rf.maxBins, [32, 64])              # Número de bins
             .build())

evaluator = MulticlassClassificationEvaluator(
    labelCol="Tipo de Piel Index", predictionCol="prediction", metricName="accuracy")

crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

cvModel = crossval.fit(train_data)

predictions2 = cvModel.transform(test_data)
accuracy2 = evaluator.evaluate(predictions2)
print("Precisión: ", accuracy2)

bestModel = cvModel.bestModel
print("Mejores hiperparámetros:")
print(" - numTrees:", bestModel.getNumTrees)
print(" - maxDepth:", bestModel._java_obj.getMaxDepth())
print(" - maxBins:", bestModel._java_obj.getMaxBins())

Precisión:  1.0
Mejores hiperparámetros:
 - numTrees: 20
 - maxDepth: 5
 - maxBins: 32


In [ ]:
# Comparar primeras predicciones con data real
predictions2.select("features", "Tipo de Piel Index", "prediction").show()

+-------------+------------------+----------+
|     features|Tipo de Piel Index|prediction|
+-------------+------------------+----------+
|[0.0,1.0,0.0]|                 2|       2.0|
|[2.0,1.0,0.0]|                 0|       0.0|
|[1.0,2.0,0.0]|                 2|       2.0|
|[0.0,2.0,0.0]|                 1|       1.0|
|[0.0,2.0,0.0]|                 1|       1.0|
+-------------+------------------+----------+



Con RandomForestClasiffier se logra una precisión de 1, es decir, todas las clasificaciones son correctas (tal como podemos ver en la comparación de la celda anterior).